### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pickle

import warnings
warnings.filterwarnings("ignore")

#### Read the train file

In [2]:
train_file = pd.read_csv('train_file.csv')

#### Display first 5 records from the file

In [3]:
train_file.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


#### Understand the structure of the datasets

In [4]:
print('The size of the train file:', train_file.shape)

The size of the train file: (614, 13)


In [5]:
#drop the null values
train_file.dropna(axis=0, inplace=True)

In [6]:
train_file.shape

(480, 13)

#### List of Preprocessing steps:                             
1) Retain only important fields 'Credit_History', 'Property_Area', 'Married' & 'LoanAmount'.      
2) Convert the field 'Married' into '0's & '1's using Label Encoder.              
3) Use one-hot encoder for the field 'Property_Area'.  
4) LoanAmount is rescaled using Standard Scaler

In [7]:
train_x = train_file[[ 'Credit_History', 'Property_Area', 'Married', 'LoanAmount']]
train_y = train_file['Loan_Status']

In [8]:
train_y = train_y.replace({'Y':1, 'N':0})

In [9]:
train_x.head()

,Credit_History,Property_Area,Married,LoanAmount
1,1.0,Rural,Yes,128.0
2,1.0,Urban,Yes,66.0
3,1.0,Urban,Yes,120.0
4,1.0,Urban,No,141.0
5,1.0,Urban,Yes,267.0


In [10]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Credit_History  480 non-null    float64
 1   Property_Area   480 non-null    object 
 2   Married         480 non-null    object 
 3   LoanAmount      480 non-null    float64
dtypes: float64(2), object(2)
memory usage: 18.8+ KB


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

numeric_features = ['LoanAmount']
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])    

categorical_features = ['Property_Area', 'Married']
categorical_transformer = OneHotEncoder(drop='first', sparse=False)

preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_transformer, categorical_features)])
                                               

In [11]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(random_state=100))])

NameError: name 'preprocessor' is not defined

In [24]:
clf.fit(train_x, train_y)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['LoanAmount']),
                                                 ('cat',
                                                  OneHotEncoder(c

In [27]:
pickle.dump(clf, open('model.pkl', 'wb'))